In [52]:
import requests
import json
import os
import time

from io import BytesIO
from functools import wraps
from pathlib import Path
from enum import Enum

import imagehash
from PIL import Image

# Config

In [61]:
API_ROUTE = "https://api.twitter.com/2/"
STREAM_ROUTE = "tweets/search/stream"
RULES_ROUTE = "tweets/search/stream/rules"
FULL_SEARCH_ROUTE = "tweets/search/all"
# ROOT_FOLDER = r"/home/tyra/Documents/Collectes"
ROOT_FOLDER = r"C:\Users\Orion\Documents\OutputTweets"
# CREDENTIALS_FOLDER = r"/home/tyra/Documents/CERES/credentials_pro.json"
CREDENTIALS_FILES = r"C:\Users\Orion\Documents\Projets\CERES\credentials_pro.json"
# maximum size of the collect in octets
MAX_SIZE = 100000000
TOKEN = generate_token()
params = {
    "tweet.fields": "public_metrics,referenced_tweets",
    "expansions": "author_id,in_reply_to_user_id,attachments.media_keys",
    "media.fields": "url",
    "user.fields": "id,verified"
}

#### Load Token

In [4]:
def generate_token():
    with open(CREDENTIALS_FILES, 'r') as f:
        return f"Bearer {json.load(f)['token']}"

In [6]:
s = requests.Session()
s.headers.update({"Authorization": TOKEN})

In [7]:
class Media(Enum):
    PHOTO = 1
    VIDEO = 2
    GIF = 3
    
    def __eq__(self, other):
        if isinstance(other, str):
            return self.name.lower() == other

In [8]:
def timeit(f):
    @wraps(f)
    def wrapper(*args, **kwargs):
        t1 = time.time()
        res = f(*args, **kwargs)
        print(f"{f.__name__} executed in {time.time() - t1}")
        return res
    return wrapper

In [ ]:
# test_handle_tweet
tweet = """
{
    "data": {
        "attachments": {
            "media_keys": [
                "3_1437787186561228800"
            ]
        },
        "author_id": "21313364",
        "entities": {
            "urls": [
                {
                    "start": 129,
                    "end": 152,
                    "url": "https://t.co/1kUyRMrCmS",
                    "expanded_url": "https://www.nouvelobs.com/droits-des-femmes/20210914.OBS48633/il-y-a-un-afflux-considerable-de-demandes-le-gouvernement-veut-reduire-les-delais-d-attente-pour-la-pma-d-un-an-a-six-mois.html?utm_term=Autofeed&utm_medium=Social&utm_source=Twitter#Echobox=1631630086",
                    "display_url": "nouvelobs.com/droits-des-fem…",
                    "images": [
                        {
                            "url": "https://pbs.twimg.com/news_img/1437787191103692800/Wg5T_O5D?format=jpg&name=orig",
                            "width": 1024,
                            "height": 512
                        },
                        {
                            "url": "https://pbs.twimg.com/news_img/1437787191103692800/Wg5T_O5D?format=jpg&name=150x150",
                            "width": 150,
                            "height": 150
                        }
                    ],
                    "status": 200,
                    "title": "« Il y a un afflux considérable de demandes » : le gouvernement veut réduire les délais d’attente pour la PMA d’un an à six mois",
                    "description": "Le Parlement a voté fin juin la procréation médicalement assistée pour toutes les femmes, qu’elles soient hétérosexuelles, homosexuelles ou monoparentales. Une enveloppe de 8 millions d’euros doit permettre de répondre à l’afflux de demandes.",
                    "unwound_url": "https://www.nouvelobs.com/droits-des-femmes/20210914.OBS48633/il-y-a-un-afflux-considerable-de-demandes-le-gouvernement-veut-reduire-les-delais-d-attente-pour-la-pma-d-un-an-a-six-mois.html?utm_term=Autofeed&utm_medium=Social&utm_source=Twitter#Echobox=1631630086"
                },
                {
                    "start": 153,
                    "end": 176,
                    "url": "https://t.co/1uGLGEP38c",
                    "expanded_url": "https://twitter.com/lobs/status/1437787188868091906/photo/1",
                    "display_url": "pic.twitter.com/1uGLGEP38c"
                }
            ]
        },
        "id": "1437787188868091906",
        "organic_metrics": {},
        "text": "« Il y a un afflux considérable de demandes » : le gouvernement veut réduire les délais d’attente pour la PMA d’un an à six mois https://t.co/1kUyRMrCmS https://t.co/1uGLGEP38c"
    },
    "includes": {
        "media": [
            {
                "media_key": "3_1437787186561228800",
                "type": "photo",
                "url": "https://pbs.twimg.com/media/E_QL1BWXIAAEEPV.jpg"
            }
        ],
        "users": [
            {
                "id": "21313364",
                "name": "L'Obs",
                "username": "lobs"
            }
        ]
    },
    "matching_rules": [
        {
            "id": 1437707103754506241,
            "tag": "PMA"
        }
    ]
}"""
handle_tweet(json.loads(tweet))


In [45]:
def handle_media(tweet):
    # check if there are some media in the tweet
    if not tweet.get('includes', {}).get('media', False):
        return
    # extract the media
    media = tweet['includes']['media']
    # TODO: use concurrent to download simultanously or even in fire and forget
    for medium in media:
        if medium['type'] == Media.PHOTO:
            media_url = medium['url']
        else:
            print(f"unhandled media type currently: {medium['type']}")
            continue
        # if we already have stored this media (avoid to redownload in case of retweet)
        if medium['media_key'] in [x.split('.')[0] for x in os.listdir(os.path.join(ROOT_FOLDER, 'media'))]:
            print("Media already downloaded")
            pass
        else:
            download_media(**medium)

In [55]:
def compute_signature(buffer):
    img = Image.open(BytesIO(buffer))
    return str(imagehash.average_hash(img))

In [59]:
def download_media(media_key=None, url=None, **kwargs):
    if not media_key or not url:
        raise ValueError("Missing field when trying to save media")
    file_type = url.split('.')[-1]
    file_name = f"{media_key}.{file_type}"
    
    # download the file
    try:
        res = requests.get(url)
        print("downloading media")
    except requests.RequestException:
        raise ValueError(f"There was an error when downloading the media with following url: {url}, please check your connection or url")
    
    # calculate signature of content, if this signature already exists then just increment the number of 
    buffer = res.content
    signature = compute_signature(buffer)
    if signature in os.listdir(os.path.join(ROOT_FOLDER, 'media')):
        
        # lets find the id of the identical file, already saved as an empty file under media/<signature>/id
        identical_media_key = os.listdir(os.path.join(ROOT_FOLDER, 'media', signature))[0]
        
        print(f"a media with the same signature already exists: {identical_media_key}")
        
        # we need to save an empty file to know how to find the existing image from the media_key
        with open(os.path.join(ROOT_FOLDER, 'media', f"{media_key}.{identical_media_key}"), 'w') as f:
            f.write('')
        return
    else:
        path = os.path.join(ROOT_FOLDER, 'media', signature)
        Path(path).mkdir(exist_ok=True, parents=True)
        with open(os.path.join(path, media_key), 'w') as f:
            # create an empty file named with the id so we know this signature = this id
            f.write('')
    # save to disk
    with open(os.path.join(ROOT_FOLDER, 'media', file_name), 'wb') as f:
        f.write(res.content)

In [12]:
@timeit
def handle_tweet(tweet):
    id = tweet['data']['id']
    
    # filter tags:
    tags = list(set([r['tag'] for r in tweet["matching_rules"]]))
    
    # save media:
    handle_media(tweet)
    
    # save collected tweet in every tag
    for tag in tags:
        # create directory if not exist
        directory = Path(os.path.join(ROOT_FOLDER), tag)
        directory.mkdir(parents=True, exist_ok=True)
        with open(os.path.join(directory, f"{id}.json"), 'w', encoding='utf-8') as f:
            json.dump(tweet, f, indent=4, ensure_ascii=False)

In [13]:
def get_folder_size(path):
    return sum(f.stat().st_size for f in Path(path).glob('**/*') if f.is_file())

In [14]:
@timeit
def has_free_space():
    if get_folder_size(ROOT_FOLDER) < MAX_SIZE:
        return True
    raise OSError("The maxsize of the storage directory has been reached")

In [16]:
def get_rules():
    """
    Allow to get all rules currently active for the collect
    """
    rules = s.get(API_ROUTE + RULES_ROUTE)
    return rules.json()

In [25]:
def get_tags_from_rules(rules):
    if 'data' not in rules:
        raise ValueError('You seem to have no rules configured yet, please make sure to create some')
    return list(set([r['tag'] for r in rules['data']]))

In [18]:
def init_storages(folders=[]):
    """
    Ensure folders are properly created at the begining of the collect
    """
    for f in folders:
        directory = Path(os.path.join(ROOT_FOLDER, f))
        directory.mkdir(parents=True, exist_ok=True)

In [63]:
def collect():
    # first ensure all folders needed are created
    init_storages(['users', 'media', *get_tags_from_rules(get_rules())])
    
    # then connect to the stream
    with s.get(API_ROUTE + STREAM_ROUTE, params=params, stream=True, timeout=5000) as resp:
        if resp.status_code != 200:
            print(f"error {resp.status_code}")
            print(resp.content)
        for line in resp.iter_lines():
            if line and has_free_space():
                data = json.loads(line.decode("utf-8"))
                if 'data' not in data:
                    print(data)
                print(data['data']['text'])
                print(data['data']['id'])
                handle_tweet(data)
            else:
                print("waiting for new tweets")

In [64]:
collect()

has_free_space executed in 0.17099976539611816
RT @liamsdem12: Admirateur de R Camus, Zemmour défend depuis des années la thèse du « grand remplacement »,popularisé par le théoricien d’e…
1439975735565897732
handle_tweet executed in 0.0010004043579101562
has_free_space executed in 0.10555553436279297
@debatdecole1 Oui je suis d accord
1439975737654718469
handle_tweet executed in 0.00600123405456543
waiting for new tweets
waiting for new tweets
waiting for new tweets
has_free_space executed in 0.07860279083251953
RT @liamsdem12: Admirateur de R Camus, Zemmour défend depuis des années la thèse du « grand remplacement »,popularisé par le théoricien d’e…
1439975992404189184
handle_tweet executed in 0.0019996166229248047
has_free_space executed in 0.07996940612792969
RT @slpng_giants_fr: Le #terroriste de #SanDiego a commis plus de 100 crimes motivés par la haine.

Son manifeste ignoble, trouvé suite à l…
1439976009114263557
handle_tweet executed in 0.0010023117065429688
waiting for new twe

waiting for new tweets
has_free_space executed in 0.5850005149841309
@PhOlivierRN immigration massive ils faut les renvoyer dans leurs pays et vite ✈️
1439978418410557444
handle_tweet executed in 0.004990100860595703
waiting for new tweets
waiting for new tweets
has_free_space executed in 0.4719996452331543
@AlSeg95 @BragueDe Il ne parle pas assez des chiffres et des conséquences du GR (selon moi, je le suis pas tout le temps)
Et dénoncer les élites ne doit pas vouloir dire ne pas dénoncer ce qu'elles ont engendre, et vice versa. Nuances vous
1439978576443371527
handle_tweet executed in 0.0019991397857666016
has_free_space executed in 0.259993314743042
Si l’expression Grand Remplacement, vous donne de l’urticaire, parlez de Grand Basculement.
https://t.co/zEkmtnd9Gy via @causeur
1439978613475102720
handle_tweet executed in 0.0009999275207519531
waiting for new tweets
has_free_space executed in 0.487032413482666
@NouveauMonde15 @PEP03011988 @ZemmourEric Un mal pour un bien ? Tu te soume

has_free_space executed in 0.39412879943847656
On peut les tordre dans tous les sens, se boucher le nez et répéter « extrême droite » en guise de mantra conjuratoire, les chiffres prouvent bel et bien que le #GrandRemplacement n’est pas un fantasme, mais une réalité‼️
➡️ #Zemmour2022
 https://t.co/O2PIvzfnsM
1439980776678989827
handle_tweet executed in 0.0009982585906982422
waiting for new tweets
waiting for new tweets
has_free_space executed in 0.14799928665161133
RT @holste_max: Ce matin je suis passé devant une école primaire, des quartiers ouest de Nice. C'était à l'heure de l'entrée des élèves, ac…
1439980932971249665
handle_tweet executed in 0.0040013790130615234
waiting for new tweets
has_free_space executed in 0.15999174118041992
@lou_noaa @Concomb09900009 @Bardon15186794 @ZemmourEric Mais sérieux le négatif en QI, pourquoi vous traiter H24 les personnes qui ne pense pas comme vous, qui n'attisent pas la haine de l'autre comme vous?
Vous avez que ça à la bouche!
Je suis Apo esp

waiting for new tweets
has_free_space executed in 0.3921239376068115
RT @Amlei6: Observez quand vous sortez…et vous verrez…👀
#grandremplacement #cancelculture
1439983849707036676
handle_tweet executed in 0.002013683319091797
has_free_space executed in 0.15200114250183105
RT @CocardeEtud: Le magazine @Causeur a excité la bien-pensance avec sa Une sur le Grand Remplacement, mais concède que le problème est pur…
1439983883177668620
handle_tweet executed in 0.0009872913360595703
has_free_space executed in 0.4105203151702881
RT @holste_max: Ce matin je suis passé devant une école primaire, des quartiers ouest de Nice. C'était à l'heure de l'entrée des élèves, ac…
1439983900894322695
handle_tweet executed in 0.0029997825622558594
waiting for new tweets
has_free_space executed in 0.3170011043548584
@morandiniblog POURQUOI NE PEUT ON PAS VOIR LE REPLAY DE L'EMISSION D'AUJOURD'HUI ??????
1439983921551323137
handle_tweet executed in 0.0059969425201416016
has_free_space executed in 0.191509485244

waiting for new tweets
has_free_space executed in 0.09199929237365723
C’est bien ce que je dis… vous aussi, faites donc bien partie de ces décérébrés ! 
Vous dites partir au combat avec Zemmour en tête…? 
Vous avez juste besoin de massages colorectaux pendant 5 ans de plus !

Concernant mon âge…, je vous mettrais bien petite fessée au passage! https://t.co/ZO45zW073V
1439987454690349058
handle_tweet executed in 0.0010004043579101562
waiting for new tweets
has_free_space executed in 0.09999752044677734
RT @ELevyCauseur: "On peut débattre de tout sauf des chiffres" @Causeur https://t.co/eZqQxRsPG1
1439987603261083651
handle_tweet executed in 0.0009970664978027344
waiting for new tweets
has_free_space executed in 0.08702993392944336
RT @soulbrotherMDL: @edwyplenel M. Plenel, avec tout le respect que je vs dois, je lis depuis 10 ans vos "parti pris", livres
Cela n'a pas…
1439987668511862785
handle_tweet executed in 0.0019707679748535156
waiting for new tweets
has_free_space executed in 0.1

waiting for new tweets
has_free_space executed in 0.10800933837890625
@Boitealexandre @MilanPrados1 Alors que celui qui parle de grand remplacement en pointant des ethnies &amp; 1 religion du doigt pour mieux nier leur nationalité, refuse la diversité car il la considère comme dangereuse &amp; délétère, veux imposer une liste de prénoms authorisés,  a des plaintes d'abus sexuels au [2]
1439991040044257284
handle_tweet executed in 0.0020003318786621094
waiting for new tweets
has_free_space executed in 0.10795283317565918
@Amlei6 Peut on y voir un parallèle avec la facilité de certains à trouver le chemin de la Caf ?
1439991113935474701
handle_tweet executed in 0.002015829086303711
has_free_space executed in 0.07999968528747559
@Amlei6 Voilà pourquoi il doit se présenter et que nous devons voter pour lui https://t.co/8rcajSFf0a
1439991127696986112
downloading media
handle_tweet executed in 0.8909912109375
waiting for new tweets
has_free_space executed in 0.12456226348876953
La cocarde qu

has_free_space executed in 0.1080012321472168
RT @PSouveraine: Éric Zemmour est le seul capable de rassembler les Français derrière un objectif majeur de nature existentielle : sauver l…
1439993190623350792
handle_tweet executed in 0.0016047954559326172
has_free_space executed in 0.09105157852172852
RT @slpng_giants_fr: Le #terroriste de #SanDiego a commis plus de 100 crimes motivés par la haine.

Son manifeste ignoble, trouvé suite à l…
1439993214467985414
handle_tweet executed in 0.0009982585906982422
has_free_space executed in 0.13297224044799805
RT @Sunrise_Europe: 🌊 Zemmour vs Mélenchon, le Grand Remplacement encore nié par les obscurantistes gauchistes… qui sombrent dans la folie…
1439993226157510659
handle_tweet executed in 0.0010018348693847656
has_free_space executed in 0.0899953842163086
RT @Amlei6: Les burquas /hijabs s’assimilent de plus en plus en France 🇫🇷 et de plus en plus vite 😬
#paris #IledeFrance #StopImmigration  #…
1439993236228030466
handle_tweet executed in 0.002

KeyboardInterrupt: 